# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4



Download (0 bytes) Requesting

Download [                              ] (0/315276 bytes) Receiving

Download [=                             ] (8192/315276 bytes) Receiving

Download [#-                            ] (16384/315276 bytes) Receiving

Download [##-                           ] (24576/315276 bytes) Receiving

Download [###                           ] (32768/315276 bytes) Receiving

Download [###=                          ] (40960/315276 bytes) Receiving

Download [####=                         ] (49152/315276 bytes) Receiving

Download [#####-                        ] (57344/315276 bytes) Receiving

Download [######                        ] (65536/315276 bytes) Receiving

Download [#######                       ] (73728/315276 bytes) Receiving

Download [#######=                      ] (81920/315276 bytes) Receiving

Download [########-                     ] (90112/315276 bytes) Receiving

Download [#########-                    ] (98304/315276 bytes) Receiving

Download [

In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

  Created wheel for hebrew-tokenizer: filename=hebrew_tokenizer-2.3.0-py3-none-any.whl size=13416 sha256=605c4b222d10855903eec7aab886ca37ca7944c8e9d12a75b54ec29d866cf7d7
  Stored in directory: c:\users\lab102\appdata\local\pip\cache\wheels\48\4a\84\4c7085a074b805492d1c24c171cbc50667f8f61d15500bb260
Successfully built hebrew-tokenizer


In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht


C:\Users\lab102


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape


,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
# YOUR CODE HERE
def create_tokens(string):
    hebrew_tokens = ht.tokenize(string)
    hebrew_tokens_list_tuples = list(filter(lambda x: (x[0]=='HEBREW'), hebrew_tokens))
    hebrew_tokens_list = [tuple[1] for tuple in hebrew_tokens_list_tuples]
    return hebrew_tokens_list

In [26]:
# YOUR CODE HERE  
text_clf =Pipeline([
    ('vect', CountVectorizer(tokenizer=create_tokens,max_df=1.8, max_features=10000, ngram_range=(1, 2), min_df=0.0)),
    ('norm', StandardScaler(with_mean=False, copy=True)),
    ('clf_nb', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-5, random_state=42, max_iter=5, tol=None)),
])

In [46]:
# YOUR CODE HERE
# text_clf2 = Pipeline([
#     ('vect', TfidfVectorizer(tokenizer=create_tokens,norm='l2', max_features=10000, ngram_range=(1, 2), min_df=2)),
#     ('norm', StandardScaler(with_mean=False, copy=True)),
#     ('clf_nb', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42, max_iter=5, tol=None)),
# ])
# text_clf1 = Pipeline([
#     ('vect', TfidfVectorizer(tokenizer=create_tokens,norm='l2', max_features=10000,  ngram_range=(1,4), min_df=2)),
#     ('norm', preprocessing.Normalizer()),
#     ('clf', Perceptron(tol=1e-3, random_state=42, alpha=1e-4, max_iter=5)),
# ])

In [50]:
# Add as many code cells as you need
# YOUR CODE HERE
X_train=df_train.copy()
y_train=df_train['gender']
text_clf.fit(X_train['story'],y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=1.8, max_features=10000, min_df=0.0,
                                 ngram_range=(1, 2),
                                 tokenizer=<function create_tokens at 0x000002294D028550>)),
                ('norm', StandardScaler(with_mean=False)),
                ('clf_nb',
                 SGDClassifier(alpha=1e-05, max_iter=5, random_state=42,
                               tol=None))])

In [29]:
def avarage_muth(l):
    return sum(l)/len(l)


In [48]:

cros_val_info2=cross_val_score(text_clf,X_train['story'],y_train,cv=10,scoring='f1_macro')
avg2=avarage_muth(cros_val_info2)
print(avg2)

0.7122341432170488


In [52]:
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')
predicted_genders = text_clf.predict(df_test['story'])
df_test['predicted_category'] = predicted_genders
df_predicted = df_test.drop('story', axis=1)

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [53]:
df_predicted.to_csv('classification_results.csv',index=False)